---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [3]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

In [4]:
def answer_one():
    return df['Gold'].idxmax()

In [5]:
def answer_two():
    return df[['Gold','Gold.1']].max(1).idxmax()

In [6]:
def answer_three():
    df_new=df[(df['Gold'] >= 1) & (df['Gold.1'] >= 1)][['Gold','Gold.1','Gold.2']]
    return df_new[['Gold','Gold.1']].max(1).divide(df_new['Gold.2'], axis='rows').idxmax()

In [7]:
def answer_four():
    df['Points'] = pd.Series(df['Gold.2'])*3 + pd.Series(df['Silver.2'])*2 + pd.Series(df['Bronze.2'])*1 
    return df['Points']

In [8]:
def answer_five():
    # create a new df which have 3 required columns
    new_df = census_df[['SUMLEV','STNAME','CTYNAME']]
    
    # create a unique state list 
    state_list = census_df[census_df['SUMLEV']==40][['STNAME']]
    
    # create a df with the state list and county count
    state_df = pd.DataFrame(state_list)
    
    # assign all value of county count to zero
    state_df['county_count']=0
    
    # set index of state_df to state names
    state_df.set_index('STNAME', inplace=True)
    
    # iterate over each state and get the count of city name and assign it to county count of each state
    for state in state_df.index:
        countycount = new_df[new_df['STNAME'] == state]['CTYNAME'].count()
        state_df['county_count'].loc[state] = countycount
    
    # get the index of maximum county count as string
    return state_df['county_count'].idxmax()

In [9]:
 
def answer_six():
    only_counties = census_df.where(census_df['SUMLEV'] == 50)
    only_counties = only_counties.dropna()
    columns_to_keep = ['STNAME', 'CTYNAME', 'CENSUS2010POP']
    only_counties = only_counties[columns_to_keep]
    only_counties = only_counties.set_index('STNAME')
    only_counties.sort_values('CENSUS2010POP', ascending=False, inplace=True)
    d = {}
    for i in range(len(only_counties)):
        state = only_counties.iloc[i].name
        try:
            if len(d[state]) < 3:
                d[state].append(only_counties.iloc[i]['CENSUS2010POP'])
        except KeyError:
            d[state] = []
            d[state].append(only_counties.iloc[i]['CENSUS2010POP'])
    for state in d:
        d[state] = sum(d[state])
    ans = []
    newdf = pd.DataFrame(d, index=range(0,len(d)))
    newdf = newdf.T[0]
    newdf = newdf.sort_values(ascending=False)
    for i in range(3):
        ans.append(newdf[newdf == newdf.iloc[i]].index[0])
    return ans

In [10]:
def answer_six():
    # create a new df which have 3 required columns    
    new_df = census_df[['STNAME','CTYNAME','CENSUS2010POP']]

    # create a unique state list 
    state_list = census_df[census_df['SUMLEV']==40][['STNAME']]

    # create a df with the state list and population
    state_df = pd.DataFrame(state_list)
    # assign population to zero for all states
    state_df['population'] = 0

    # set index of state_df to state names
    state_df.set_index('STNAME', inplace=True)

    # get top 3 populous city for each state   

    for state in state_df.index:
        stsum=0

         # get top 3 populous city for each state                 
        top3city_pop = new_df[new_df['STNAME'] == state].sort_values(by='CENSUS2010POP', ascending=False)['CENSUS2010POP']
        if type(top3city_pop) == pd.Series:
          stsum = sum(top3city_pop[0:3])
        else:
          stsum = top3city_pop[0]
        # get sum of top 3 city and store in population for each state
        state_df['population'].loc[state] = stsum
    
    #state_df
    state_df = state_df.reset_index()
    state_df = state_df.sort_values(by='population', ascending=False)


    return state_df['STNAME'][0:3].tolist()

In [11]:
def answer_seven():
    only_counties = census_df.where(census_df['SUMLEV'] == 50)
    only_counties = only_counties.dropna()
    columns_to_keep = ['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 
                       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']
    only_counties = only_counties[columns_to_keep]
    only_counties = only_counties.set_index('CTYNAME')
    amax = 0
    ans = ""
    for i in range(len(only_counties)):
        popl = []
        popl.append(only_counties.iloc[i]['POPESTIMATE2010'])
        popl.append(only_counties.iloc[i]['POPESTIMATE2011'])
        popl.append(only_counties.iloc[i]['POPESTIMATE2012'])
        popl.append(only_counties.iloc[i]['POPESTIMATE2013'])
        popl.append(only_counties.iloc[i]['POPESTIMATE2014'])
        popl.append(only_counties.iloc[i]['POPESTIMATE2015'])
        diff = max(popl) - min(popl)
        if amax < diff:
            amax = diff
            ans = only_counties.iloc[i].name
    return ans

In [12]:
def answer_eight():
    return census_df[((census_df['REGION']==1) | (census_df['REGION']==2)) & 
                     (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014']) & 
                     census_df['CTYNAME'].str.startswith('Washington')][['STNAME','CTYNAME']]